In [11]:
import arcpy,os

scratch = r"C:\ArcPro_Projects\BurnSevMaps\BurnSevMaps.gdb"
fc = "interim_burn_severity_2025"
out_table = os.path.join(scratch,fc+'_SummarizeAttributes')

aprx = arcpy.mp.ArcGISProject("CURRENT")
mymap = aprx.listMaps('Burn Severity')[0]

arcpy.gapro.SummarizeAttributes(
    input_layer=fc,
    out_table=out_table,
    fields="FIRE_NUMBER",
    summary_fields="PRE_FIRE_IMAGE_DATE ANY;POST_FIRE_IMAGE_DATE ANY",
    time_step_interval=None,
    time_step_repeat=None,
    time_step_reference=None
)

<Result 'C:\\ArcPro_Projects\\BurnSevMaps\\BurnSevMaps.gdb\\interim_burn_severity_2025_SummarizeAttributes'>

In [16]:
## Calculate PRE_IMG and POST_IMG dates
arcpy.management.CalculateField(
    in_table=out_table,
    field="PRE_IMG",
    expression="!ANY_PRE_FIRE_IMAGE_DATE!",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table=out_table,
    field="POST_IMG",
    expression="!ANY_POST_FIRE_IMAGE_DATE!",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

# Join the two fields to the perimeters
arcpy.management.JoinField(
    in_data="Fire Perimeters",
    in_field="FireNum",
    join_table=out_table,
    join_field="FIRE_NUMBER",
    fields="PRE_IMG;POST_IMG",
    fm_option="NOT_USE_FM",
    field_mapping=None,
    index_join_fields="NO_INDEXES"
)

mymap.clearSelection()